# Overview
<p>Periodic loading of Peer to Peer financing data to an SQLite database. Operating for:
<br>--Mintos
<br>--IUVO
</p>

In [41]:
import pandas as pd
import matplotlib.pyplot as plt
import sqlite3 as sql
import zipfile
import os

In [15]:
#Path to local Excel files containing the downloaded data from the P2P sites
path = "/home/hpiyankov/Documents/P2P"

#Get the diretory with the latest downloads, we will use it also as date for the SQLite
dirList = [x[0] for x in os.walk(path)]
lastDir = max(dirList)
maxDate = lastDir[-10:]

print(lastDir)
print(maxDate)

/home/hpiyankov/Documents/P2P/2018-01-01
2018-01-01


# Mintos

## Loading data to dataframes

In [20]:
myZip = zipfile.ZipFile(lastDir+"/loan_book.zip","r")
contents = myZip.namelist()
print(*contents,sep="\n")

1-500000_loan_book.xlsx
500001-1000000_loan_book.xlsx
1000001-1500000_loan_book.xlsx
1500001-2000000_loan_book.xlsx
2000001-2500000_loan_book.xlsx


In [34]:
mintosDF = pd.DataFrame()

for m in contents:
    mintosDF = mintosDF.append(pd.read_excel(myZip.open(m,"r")))

mintosDF.shape

(2039464, 17)

In [37]:
#Let's make sure all is as expected
assert mintosDF.shape[0] > 100000
assert mintosDF.shape[1] == 17

In [38]:
#Drop spaces form column names for easier SQL storage
mintosDF.columns = [x.replace(" ","") for x in mintosDF.columns]
mintosDF["DATE"] = maxDate
mintosDF.head(5)

,Id,IssueDate,ListingDate,Country,LoanOriginator,LoanType,LoanRatePercent,Term,Collateral,InitialLTV,LTV,LoanStatus,Buybackreason,InitialLoanAmount,RemainingLoanAmount,Currency,Buyback,DATE
0,2014-1012,2014-10-17 00:00:00,2014-10-17 00:00:00,Latvia,Hipocredit,Mortgage Loan,14.2,60,Yes,13,6,Current,-,5000.0,2253.21,EUR,No,2018-01-01
1,17938-02,2016-02-02 00:00:00,2016-02-04 18:39:18,Latvia,Hipocredit,Mortgage Loan,15.4,60,Yes,45,41,Current,-,40525.0,36473.42,EUR,No,2018-01-01
2,2014-1003,2014-10-17 00:00:00,2014-10-17 00:00:00,Latvia,Hipocredit,Mortgage Loan,12.5,120,Yes,45,36,Late 1-15,-,30000.0,24354.30,EUR,No,2018-01-01
3,2014-10019,2014-10-10 00:00:00,2014-10-10 00:00:00,Latvia,Hipocredit,Mortgage Loan,13.6,120,Yes,60,49,Late 1-15,-,18000.0,14592.10,EUR,No,2018-01-01
4,2014-1002,2014-10-10 00:00:00,2014-10-10 00:00:00,Latvia,Hipocredit,Mortgage Loan,13.9,84,Yes,28,0,Finished prematurely,Not Specified,7700.0,0.00,EUR,No,2018-01-01


## Loading dataframe to SQLite DB

In [ ]:
con = sql.connect(path+"\Mintos_bup.db")